In [22]:
import numpy as np
import math

def read_tsp_file(file_path):
    city_coordinates = {}
    with open(file_path, 'r') as file:
        # Temukan awal dari NODE_COORD_SECTION
        for line in file:
            if line.strip() == "NODE_COORD_SECTION":
                break
        
        # Baca data koordinat kota
        for line in file:
            if line.strip() == "EOF":
                break
            # Split line and convert values to integers
            values = list(map(int, line.split()))
            
            if len(values) == 2:
                # If there are two values, assume (x, y) and use the line number as city_id
                city_id = len(city_coordinates) + 1
                x, y = values
            elif len(values) == 3:
                # If there are three values, assume (city_id, x, y)
                city_id, x, y = values
            else:
                # Handle other cases as needed
                continue
            
            city_coordinates[city_id] = (x, y)
    
    return city_coordinates

file_path = 'D:/att48.tsp'
city_coordinates = read_tsp_file(file_path)

# Function to calculate Euclidean distance
def euclidean_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Generate initial population
def generate_initial_population(city_coordinates, population_size):
    city_ids = list(city_coordinates.keys())
    population = [np.random.permutation(city_ids).tolist() for _ in range(population_size)]
    return population

# Calculate total distance of a path
def calculate_total_distance(path, city_coordinates):
    total_distance = 0
    for i in range(len(path)):
        from_city = city_coordinates[path[i]]
        to_city = city_coordinates[path[(i + 1) % len(path)]]  # Loop back to start
        total_distance += euclidean_distance(from_city, to_city)
    return total_distance

# Fitness evaluation
def evaluate_fitness(population, city_coordinates):
    fitness_scores = []
    for individual in population:
        distance = calculate_total_distance(individual, city_coordinates)
        fitness_score = 1 / distance  # Higher fitness for shorter path
        fitness_scores.append(fitness_score)
    return np.array(fitness_scores)

# Selection - Roulette Wheel Selection
def select_parents(population, fitness_scores, num_parents):
    parents = []
    for _ in range(num_parents):
        parent_idx = np.random.choice(np.arange(len(population)), p=fitness_scores/fitness_scores.sum())
        parents.append(population[parent_idx])
    return parents

# # Crossover - Ordered Crossover
# def ordered_crossover(parent1, parent2):
#     size = len(parent1)
#     start, end = sorted(np.random.sample(2) * size)
#     child = [None]*size
#     child[int(start):int(end)] = parent1[int(start):int(end)]
#     fill_values = [item for item in parent2 if item not in child]
#     child = [item if item is not None else fill_values.pop(0) for item in child]
#     return child
def two_point_crossover(parent1, parent2):
    size = len(parent1)
    # Hasilkan dua titik berbeda untuk saling bersilangan
    crossover_points = sorted(np.random.choice(range(1, size), 2, replace=False))
    start, end = crossover_points
    
    # Initialize the child with None
    """
    menginisialisasi daftar bernama child dengan panjang sama dengan size, dimana setiap elemen dalam daftar awalnya disetel ke None.
    Menginisialisasi [None] dapat membantu memeriksa error. 
    Setelah operasi crossover selesai, jika ada nilai None yang tersisa, hal ini menunjukkan bahwa algoritma 
    tidak mengisi array child dengan benar, yang dapat membantu dalam men-debug proses crossover.
    """
    child = [None] * size
    
    # Salin segmen dari parent1 ke anak
    child[start:end] = parent1[start:end]
    # Isi sisa rute dari induk2, pastikan tidak ada duplikat
    current_position = end
    for city in parent2:
        if city not in child:
            if current_position >= size:
                current_position = 0
            child[current_position] = city
            current_position += 1
    
    return child


# Mutation - Swap Mutation
def swap_mutation(individual, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(len(individual), 2, replace=False)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Jalankan algoritma genetika untuk TSP
def genetic_algorithm_tsp_with_elitism(city_coordinates, population_size=100, num_generations=500, mutation_rate=0.01, elitism_size=2):
    population = generate_initial_population(city_coordinates, population_size)
    for generation in range(num_generations):
        fitness_scores = evaluate_fitness(population, city_coordinates)
        total_fitness = np.sum(fitness_scores)
        best_distance = 1 / np.max(fitness_scores)
        
        # Elitisme: Cadangan individu terbaik dari populasi saat ini
        elite_indices = np.argsort(-fitness_scores)[:elitism_size]
        elites = [population[i] for i in elite_indices]
        
        # Hasilkan populasi berikutnya, tidak termasuk elit untuk saat ini
        next_population = []
        parents = select_parents(population, fitness_scores, population_size - elitism_size)
        for i in range(0, len(parents), 2):
            if i+1 < len(parents):  # Ensure there's a pair for crossover
                for child in [ordered_crossover(parents[i], parents[i+1]), ordered_crossover(parents[i+1], parents[i])]:
                    child = swap_mutation(child, mutation_rate)
                    next_population.append(child)
        
        # Tambahkan elit ke populasi berikutnya setelah mutasi
        next_population.extend(elites)
        
        population = next_population
        
        # Perbarui skor kebugaran berdasarkan populasi baru termasuk elit
        fitness_scores = evaluate_fitness(population, city_coordinates)
        best_distance = 1 / np.max(fitness_scores)  # Hitung ulang jarak terbaik dengan menyertakan elit
        
        # Display generation info
        print(f"Generasi {generation + 1}\nBest Distance = {best_distance}\nTotal Fitness = {total_fitness}\n")

    # Identifikasi solusi terbaik dalam populasi akhir
    best_idx = np.argmax(fitness_scores)
    best_solution = population[best_idx]
    return best_solution, best_distance

# Jalankan Algoritma Genetika dengan elitisme dan tampilkan hasilnya
best_path, best_distance = genetic_algorithm_tsp_with_elitism(city_coordinates)
print(f"\nBest Path:{best_path}\nBest Distance: {best_distance}")

Generasi 1
Best Distance = 128998.17367022487
Total Fitness = 0.0006351994035939977

Generasi 2
Best Distance = 128998.17367022487
Total Fitness = 0.0006444708899463422

Generasi 3
Best Distance = 128998.17367022487
Total Fitness = 0.0006487037740072894

Generasi 4
Best Distance = 125885.91881119314
Total Fitness = 0.0006503304525806564

Generasi 5
Best Distance = 125885.91881119314
Total Fitness = 0.0006567608781710531

Generasi 6
Best Distance = 125885.91881119314
Total Fitness = 0.0006628611263908592

Generasi 7
Best Distance = 125801.36679706605
Total Fitness = 0.0006548592877155638

Generasi 8
Best Distance = 125801.36679706605
Total Fitness = 0.0006564817472789036

Generasi 9
Best Distance = 125801.36679706605
Total Fitness = 0.0006538510398549595

Generasi 10
Best Distance = 125801.36679706605
Total Fitness = 0.0006551091545230451

Generasi 11
Best Distance = 125801.36679706605
Total Fitness = 0.0006549977133654062

Generasi 12
Best Distance = 125801.36679706605
Total Fitness = 

Generasi 106
Best Distance = 96710.09531269931
Total Fitness = 0.0007282099269288009

Generasi 107
Best Distance = 96710.09531269931
Total Fitness = 0.0007370048044303167

Generasi 108
Best Distance = 96710.09531269931
Total Fitness = 0.0007328184149193029

Generasi 109
Best Distance = 96710.09531269931
Total Fitness = 0.0007502046302480612

Generasi 110
Best Distance = 96710.09531269931
Total Fitness = 0.0007365176052220494

Generasi 111
Best Distance = 96710.09531269931
Total Fitness = 0.0007400243914458996

Generasi 112
Best Distance = 96710.09531269931
Total Fitness = 0.000743250121650488

Generasi 113
Best Distance = 96710.09531269931
Total Fitness = 0.0007358596789373533

Generasi 114
Best Distance = 96710.09531269931
Total Fitness = 0.0007394547296145312

Generasi 115
Best Distance = 96710.09531269931
Total Fitness = 0.0007303605458025959

Generasi 116
Best Distance = 94075.89645224444
Total Fitness = 0.0007188336608965483

Generasi 117
Best Distance = 94075.89645224444
Total Fi

Generasi 223
Best Distance = 85905.55727871039
Total Fitness = 0.0008427900199488319

Generasi 224
Best Distance = 85905.55727871039
Total Fitness = 0.0008432930015358715

Generasi 225
Best Distance = 85905.55727871039
Total Fitness = 0.0008453984104273126

Generasi 226
Best Distance = 85905.55727871039
Total Fitness = 0.0008550812069472848

Generasi 227
Best Distance = 85905.55727871039
Total Fitness = 0.0008649594344243488

Generasi 228
Best Distance = 85905.55727871039
Total Fitness = 0.0008775580306870758

Generasi 229
Best Distance = 85905.55727871039
Total Fitness = 0.000890226247819402

Generasi 230
Best Distance = 85905.55727871039
Total Fitness = 0.0008664431298079011

Generasi 231
Best Distance = 85905.55727871039
Total Fitness = 0.0008472687180530234

Generasi 232
Best Distance = 85785.09287518167
Total Fitness = 0.0008183374872276632

Generasi 233
Best Distance = 85785.09287518167
Total Fitness = 0.0008244893763372044

Generasi 234
Best Distance = 85785.09287518167
Total Fi

Generasi 339
Best Distance = 69218.03865328203
Total Fitness = 0.0008719480438579328

Generasi 340
Best Distance = 69218.03865328203
Total Fitness = 0.0009285816266952364

Generasi 341
Best Distance = 69218.03865328203
Total Fitness = 0.0009572773397241818

Generasi 342
Best Distance = 68055.14221541124
Total Fitness = 0.0009764764442417752

Generasi 343
Best Distance = 68055.14221541124
Total Fitness = 0.000997495395512556

Generasi 344
Best Distance = 66487.31645883578
Total Fitness = 0.0010201909217614571

Generasi 345
Best Distance = 65786.32020218951
Total Fitness = 0.0010173575733535647

Generasi 346
Best Distance = 65786.32020218951
Total Fitness = 0.0010143088996058141

Generasi 347
Best Distance = 65786.32020218951
Total Fitness = 0.0009695978878277176

Generasi 348
Best Distance = 65786.32020218951
Total Fitness = 0.0009627125146150048

Generasi 349
Best Distance = 65786.32020218951
Total Fitness = 0.0009568301695504676

Generasi 350
Best Distance = 65786.32020218951
Total Fi

Generasi 456
Best Distance = 58644.73805600138
Total Fitness = 0.0009882941441284067

Generasi 457
Best Distance = 58644.73805600138
Total Fitness = 0.0010058465583204664

Generasi 458
Best Distance = 58644.73805600138
Total Fitness = 0.0009809287505674355

Generasi 459
Best Distance = 58644.73805600138
Total Fitness = 0.0009891359454309743

Generasi 460
Best Distance = 58644.73805600138
Total Fitness = 0.0009686813944277414

Generasi 461
Best Distance = 58644.73805600138
Total Fitness = 0.0010097987455188794

Generasi 462
Best Distance = 58644.73805600138
Total Fitness = 0.0009915249096366617

Generasi 463
Best Distance = 58644.73805600138
Total Fitness = 0.0010091070625518484

Generasi 464
Best Distance = 58644.73805600138
Total Fitness = 0.0009931318977188016

Generasi 465
Best Distance = 58644.73805600138
Total Fitness = 0.001001203897873805

Generasi 466
Best Distance = 58644.73805600138
Total Fitness = 0.001024261239562321

Generasi 467
Best Distance = 58644.73805600138
Total Fit